<a href="https://colab.research.google.com/github/hagigat/NeuralNetwork/blob/master/SOM_RBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math
import scipy.spatial.distance as sc
from sklearn.metrics import accuracy_score

In [0]:
data = pd.read_csv("pima_indians_diabetes.csv", header=None)
data = data.dropna().to_numpy()
X_train = data[0:650, 0:8]
Y_train = data[0:650, 8]
X_test = data[650:760, 0:8]
Y_test = data[650:760, 8]

In [0]:
class RBF:
    def __init__(self, X, Y):
        self.nl = 0
        self.X = self.normalize(X)
        self.Y = Y
        self.C = []
        self.weight = []
        self.std = self.deviation()

    def deviation(self):
        return np.amax(sc.pdist(self.X, 'euclidean')) / np.sqrt(2 * self.X.shape[1])

    def Gaussian(self,):
        ed = sc.cdist(self.X, self.C, 'euclidean')
        return np.exp( -0.5 * ((ed ** 2) / (self.std ** 2)))

    def tGaussian(self,X):
        ed = sc.cdist(X, self.C, 'euclidean')
        return np.exp( -0.5 * ((ed ** 2) / (self.std ** 2)))

    def nGaussian(self, a ,b):
        ed = sc.euclidean(a, b)
        return np.exp( -0.5 * ((ed ** 2) / (self.std ** 2)))        

    def w_opt(self, O, D):
        tmp = np.linalg.inv(np.dot(O.T, O))
        self.weight = np.dot(tmp, np.dot(O.T, D))

    def normalize(self, X):
        for i in range(X.shape[1]):
            X[:, i] = (X[:, i] - X[:, i].mean()) / np.std(X[:, i])
        return X

    def SOM(self, N):
        etha = 1.01
        win = np.zeros((N,N))
        R = int(N/2 + 1) 
        w = np.random.random((N, N, self.X.shape[1]))
        while R!=0:
          R = R -1
          for i in range(10*self.X.shape[0]):
            idx = np.random.randint(0, (self.X.shape[0] -1))
            row = self.X[idx]
            for j in range(w.shape[0]):
              for k in range(w.shape[1]):
                win[j][k] = self.nGaussian(row , w[j][k])
            ind = np.unravel_index(np.argmax(win, axis=None), win.shape)
            imin = ind[0] - R if ind[0] - R >= 0 else 0
            imax = ind[0] + R if ind[0] + R < N else N-1
            jmin = ind[1] - R if ind[1] - R >= 0 else 0
            jmax = ind[1] + R if ind[1] + R < N else N-1
            for j in range(imin,imax):
              for k in range(jmin , jmax):
                w[j][k] = w[j][k] + etha * (row - w[j][k]) 

        w = w.reshape(N*N , self.X.shape[1])
        return w


    def Train(self):
        # RBF Train
        self.C = self.SOM(N=7)
        print(self.C.shape)
        O = self.Gaussian()
        O = np.c_[O, np.ones(len(self.X))]
        print(O.shape)
        self.w_opt(O, self.Y)
        y = self.act_fun(np.dot(O, self.weight))
        mse = np.sum((y-self.Y)**2) / self.X.shape[0]
        print("-----> MSE :","%.4f" % mse," , Accuracy: ","%.1f" % (accuracy_score(self.Y, y)*100))

    def Test(self, X, Y):
        # RBF Test
        O = self.tGaussian(X)
        O = np.c_[O, np.ones(len(X))]
        y = self.act_fun(np.dot(O, self.weight))
        mse = np.sum((y-Y)**2) / X.shape[0]
        print("> MSE :","%.4f" % mse," , Accuracy: ","%.1f" % (accuracy_score(Y, y)*100))


    def act_fun(self, z):  # sigmoid
        result = 1.0 / (1.0 + np.exp(-z))
        result[result >= 0.5] = 1
        result[result < 0.5] = 0
        return result


In [13]:
rbf = RBF(X_train, Y_train)
rbf.Train()

(49, 8)
(650, 50)
-----> MSE : 0.3000  , Accuracy:  70.0


In [14]:
X_test = rbf.normalize(X_test)
rbf.Test(X_test,Y_test)

> MSE : 0.2818  , Accuracy:  71.8
